In [13]:
# pip install mlxtend

  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.1
    Uninstalling scikit-learn-0.23.1:
      Successfully uninstalled scikit-learn-0.23.1
Note: you may need to restart the kernel to use updated packages.


ERROR: scikit-learn 1.2.0 has requirement joblib>=1.1.1, but you'll have joblib 0.16.0 which is incompatible.


In [28]:
from csv import reader
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd
from mlxtend.frequent_patterns import apriori, fpgrowth, fpmax


In [10]:
groceries = []

with open('data/groceries.csv', 'r', encoding='utf-8-sig') as csvfile:
    csv_reader = reader(csvfile)
    for row in csv_reader:
        groceries.append(row)

In [12]:
groceries[0:5]

[['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups'],
 ['tropical fruit', 'yogurt', 'coffee'],
 ['whole milk'],
 ['pip fruit', 'yogurt', 'cream cheese', 'meat spreads'],
 ['other vegetables',
  'whole milk',
  'condensed milk',
  'long life bakery product']]

In [17]:
# Instantiate an encoder object
encoder = TransactionEncoder()
# Use the fit method to extract unique labels in the set
# Use the transform method to one-hot encode the transactions into a boolean
transactions = encoder.fit(groceries).transform(groceries)
transactions

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False,  True, False],
       [False, False, False, ...,  True, False, False],
       ...,
       [False, False, False, ..., False,  True, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [19]:
# Convert the transactions to a dataframe
itemsets = pd.DataFrame(transactions, columns = encoder.columns_)
itemsets.head()

,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,berries,beverages,bottled beer,bottled water,brandy,brown bread,butter,butter milk,cake bar,candles,candy,canned beer,canned fish,canned fruit,canned vegetables,cat food,cereals,chewing gum,chicken,chocolate,chocolate marshmallow,citrus fruit,cleaner,cling film/bags,cocoa drinks,coffee,condensed milk,cooking chocolate,cookware,cream,cream cheese,curd,...,sauces,sausage,seasonal products,semi-finished bread,shopping bags,skin care,sliced cheese,snack products,soap,soda,soft cheese,softener,sound storage medium,soups,sparkling wine,specialty bar,specialty cheese,specialty chocolate,specialty fat,specialty vegetables,spices,spread cheese,sugar,sweet spreads,syrup,tea,tidbits,toilet cleaner,tropical fruit,turkey,uht-milk,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False


In [20]:
# Summary of the dataframe
itemsets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9835 entries, 0 to 9834
Columns: 169 entries, abrasive cleaner to zwieback
dtypes: bool(169)
memory usage: 1.6 MB


# Generate frequent itemsets

Assume we want itemsets that occur at least 5 times a day. The data spans 30 days and has 9,835 transactions. We need to set our minimum support threshold to $5 \times \frac{30}{9835} \approx 0.015$

In [23]:
# Display the frequent itemsets with column names
frequent_itemsets = apriori(itemsets, min_support = 0.015, use_colnames = True)
frequent_itemsets

,support,itemsets
0,0.017692,(baking powder)
1,0.052466,(beef)
2,0.033249,(berries)
3,0.026029,(beverages)
4,0.080529,(bottled beer)
...,...,...
175,0.023183,"(other vegetables, root vegetables, whole milk)"
176,0.017082,"(other vegetables, whole milk, tropical fruit)"
177,0.022267,"(other vegetables, yogurt, whole milk)"
178,0.015557,"(yogurt, whole milk, rolls/buns)"


In [25]:
# Sort by support
frequent_itemsets.sort_values('support', ascending = False)

,support,itemsets
71,0.255516,(whole milk)
44,0.193493,(other vegetables)
53,0.183935,(rolls/buns)
60,0.174377,(soda)
72,0.139502,(yogurt)
...,...,...
163,0.015252,"(shopping bags, yogurt)"
179,0.015150,"(yogurt, whole milk, tropical fruit)"
45,0.015048,(pasta)
11,0.015048,(canned fish)


In [26]:
# Let's look at only frequent itemsets with a length greater than two. First get the length of the itemsets:
length = frequent_itemsets['itemsets'].str.len()
# Then create a filter:
rows = length > 2
# Finally, apply the filter to the dataframe:
frequent_itemsets[rows]

,support,itemsets
174,0.017895,"(other vegetables, whole milk, rolls/buns)"
175,0.023183,"(other vegetables, root vegetables, whole milk)"
176,0.017082,"(other vegetables, whole milk, tropical fruit)"
177,0.022267,"(other vegetables, yogurt, whole milk)"
178,0.015557,"(yogurt, whole milk, rolls/buns)"
179,0.015150,"(yogurt, whole milk, tropical fruit)"


In [27]:
frequent_itemsets.groupby(length)['support'].describe()

,count,mean,std,min,25%,50%,75%,max
itemsets,,,,,,,,
1,73.0,0.053441,0.045956,0.015048,0.024504,0.037112,0.064870,0.255516
2,101.0,0.024799,0.010058,0.015048,0.018404,0.021047,0.027555,0.074835
3,6.0,0.018522,0.003417,0.015150,0.015938,0.017489,0.021174,0.023183


Most transactions considered frequent are two-item purchases (count: 101). Single-item purchases have a higher support value (according to the mean and 50% columns) than two- and three-item purchases.

# Create association rules